In [4]:
import pandas as pd

def log_loader(logpath):
    with open(logpath) as f:
        rawtxt = f.read().replace("\x1b[94m","").replace("\x1b[91m","").replace("\x1b[0m","").replace("\x1b[92m","").split("\n")
    df = pd.DataFrame(columns=['Epoch','val_acc','test_acc'])
    df_loss = pd.DataFrame(columns=['curr_loss','sec_per_sent'])
    for i in range(len(rawtxt)):
        if "Epoch" in rawtxt[i]:
            epoch =  int(rawtxt[i][-2:])
            val_test_list = rawtxt[i+2].split(" ")
            valac = float(val_test_list[5])
            testac = float(val_test_list[10])
            df = df.append({
                'Epoch':epoch,
                'val_acc':valac,
                'test_acc':testac
            }, ignore_index=True)
        if "curr_loss" in rawtxt[i]:
            loss_list = rawtxt[i].split(" ")
            curr_loss = float(loss_list[7][:-1])
            sec_per_sent = float(loss_list[-1])
            df_loss = df_loss.append({
                'curr_loss':curr_loss,
                'sec_per_sent':sec_per_sent
            }, ignore_index=True)
    return df, df_loss

In [57]:
sample_size = 128000

In [27]:
twod_eucl_rnn, twod_eucl_rnn_loss = log_loader('2D_eucl_rnn.txt')

In [28]:
twod_eucl_rnn.loc[29]

Epoch       29.0000
val_acc      0.7763
test_acc     0.7792
Name: 29, dtype: float64

In [58]:
time_two = np.cumsum(twod_eucl_rnn_loss['sec_per_sent']*sample_size)
time_two

0        64.0
1       128.0
2       192.0
3       256.0
4       320.0
        ...  
119    6745.6
120    6809.6
121    6873.6
122    6937.6
123    7001.6
Name: sec_per_sent, Length: 124, dtype: float64

In [31]:
twod_hyp_rnn, twod_hyp_rnn_loss = log_loader('2D_hyp_rnn.txt')

In [32]:
twod_hyp_rnn.loc[22]

Epoch       22.0000
val_acc      0.7885
test_acc     0.7848
Name: 22, dtype: float64

In [59]:
time_two= np.cumsum(twod_hyp_rnn_loss['sec_per_sent']*sample_size)
time_two

0       307.2
1       627.2
2       934.4
3      1241.6
4      1548.8
       ...   
91    28057.6
92    28364.8
93    28672.0
94    28979.2
95    29286.4
Name: sec_per_sent, Length: 96, dtype: float64

In [60]:
29286./60/60

8.135

In [37]:
five_eucl_rnn, five_eucl_rnn_loss = log_loader('5D_eucl_rnn.txt')

In [38]:
five_eucl_rnn.loc[29]

Epoch       29.0000
val_acc      0.7854
test_acc     0.7812
Name: 29, dtype: float64

In [61]:
time_fived = np.cumsum(five_eucl_rnn_loss['sec_per_sent']*sample_size)
time_fived

0        51.2
1       102.4
2       153.6
3       204.8
4       256.0
        ...  
119    6144.0
120    6195.2
121    6246.4
122    6297.6
123    6348.8
Name: sec_per_sent, Length: 124, dtype: float64

In [40]:
five_hyp_rnn, five_hyp_rnn_loss = log_loader('5D_hyp_rnn.txt')

In [41]:
five_hyp_rnn.loc[29]

Epoch       6.0000
val_acc     0.8156
test_acc    0.8144
Name: 29, dtype: float64

In [63]:
time_fived = np.cumsum(five_hyp_rnn_loss['sec_per_sent']*sample_size)
time_fived

0        358.4
1        729.6
2       1100.8
3       1484.8
4       1856.0
        ...   
119    42329.6
120    42662.4
121    42995.2
122    43340.8
123    43673.6
Name: sec_per_sent, Length: 124, dtype: float64

In [43]:
ten_eucl_rnn, ten_eucl_rnn_loss = log_loader('10D_eucl_rnn.txt')

In [44]:
ten_eucl_rnn.loc[29]

Epoch       29.0000
val_acc      0.7928
test_acc     0.7782
Name: 29, dtype: float64

In [64]:
time_ten = np.cumsum(ten_eucl_rnn_loss['sec_per_sent']*sample_size)
time_ten

0        76.8
1       153.6
2       230.4
3       307.2
4       384.0
        ...  
119    9216.0
120    9292.8
121    9369.6
122    9446.4
123    9523.2
Name: sec_per_sent, Length: 124, dtype: float64

In [14]:
ten_hyp_rnn, ten_hyp_rnn_loss = log_loader('10D_hyp_rnn.txt')

In [19]:
ten_hyp_rnn.loc[29]

Epoch       3.0000
val_acc     0.8211
test_acc    0.8217
Name: 29, dtype: float64

In [65]:
time_ten = np.cumsum(ten_hyp_rnn_loss['sec_per_sent']*sample_size)
time_ten

0        307.2
1        614.4
2        921.6
3       1228.8
4       1536.0
        ...   
122    38259.2
123    38566.4
124    38886.4
125    39219.2
126    39539.2
Name: sec_per_sent, Length: 127, dtype: float64

In [66]:
39539/60/60

10.983055555555556